In [27]:
import cvxpy as cp
import numpy as np
import networkx as nx


inputnodes = [0,5]
inputedges = [(0,2),(4,5),(1,3),(0,4),(4,1),(2,5),(5,3)]
C = {
    (0,2):1,
    (4,5):1,
    (1,3):1,
    (0,4):1,
    (4,1):1,
    (2,5):1,
    (5,3):1,
}
non_member = [4,5]
source = [0,1]
recievers = [[3],[2]]

sessions = [0,1]


preG = nx.Graph()
preG.add_nodes_from(inputnodes)
preG.add_edges_from(inputedges)
directE = []
for n, nbrs in preG.adj.items():
    for nbr, eattr in nbrs.items():
        directE.append((n,nbr))


G = nx.DiGraph()
G.add_nodes_from(inputnodes)
G.add_edges_from(directE)

undirected_edges = set(inputedges)

vertices = set(np.arange(inputnodes[1]+1))

edges = set(directE)


# ----- building LP ---------------

fstar_constraints = []
flow_constraints = []
capacity_constraints = []
variable_constraints = []


phi = {(i, j): cp.Variable() for i in sessions for j in recievers[i]}   # f* == incoming reciever flow

fstar_constraints = [(-1 + sum(phi[i,j] for i in sessions for j in recievers[i])) == 0]

mu = {(i, j, a): cp.Variable() for i in sessions for j in recievers[i] for a in edges}   # fija >= 0
psi = {(i, j, a): cp.Variable() for i in sessions for j in recievers[i] for a in edges}   # fija <= fia
xi = {(a): cp.Variable() for a in edges}   # sum(fia) <= c(a)

for i in sessions: 
    for a in edges: 
        variable_constraints += [xi[a] >= 0]      # it loops more times than it needs, change?
        for j in recievers[i]:
            variable_constraints += [psi[i,j,a] >= 0]
            variable_constraints += [mu[i,j,a] >= 0]

rho = {(i, j, v): cp.Variable() for i in sessions for j in recievers[i] for v in vertices - {i} - {j}}   # outfrom non-member == into non-memebr
sigma = {(i, j): cp.Variable() for i in sessions for j in recievers[i]}   # into source == 0
tau = {(i, j): cp.Variable() for i in sessions for j in recievers[i]}   # outfrom reciever == 0

for i in sessions:
    for j in recievers[i]:
        non_member = vertices - {i} - {j}
        for a in edges:
            equation = - mu[i,j,a] + psi[i,j,a]
            if a[1] in non_member:
                equation += rho[i,j,a[1]]
            if a[0] in non_member:
                equation -= rho[i,j,a[0]]
            if a[1] == int(source[i]):
                equation += sigma[i,j]
            if a[0] == int(j):
                equation += tau[i,j]
            if a[1] == int(j):
                equation -= phi[i,j]
                
            flow_constraints += [equation == 0]
        

for a in edges:
    psi_sum = sum(psi[i,j,a] for i in sessions for j in recievers[i])
    flow_constraints += [- psi_sum + xi[a] == 0]


lambd = {a: cp.Variable() for a in edges}   
nu = {a1: cp.Variable() for a1 in undirected_edges}   # fia <= c(a)

for a1 in undirected_edges:
    a2 = (a1[1], a1[0]) 
    equation1 = nu[a1]
    equation2 = nu[a1]
    
    equation1 -= (xi[a1] + lambd[a1])
    equation2 -= (xi[a2] + lambd[a2])

    variable_constraints += [lambd[a1] >= 0]
    variable_constraints += [lambd[a2] >= 0]

    capacity_constraints += [equation1 == 0]
    capacity_constraints += [equation2 == 0]
    

objective = cp.Minimize(sum(nu[a1]*C[a1] for a1 in undirected_edges))

constraints = fstar_constraints + flow_constraints + capacity_constraints + variable_constraints

problem = cp.Problem(objective, constraints)
problem.solve()

print("minimal objective:", sum(nu[a1]*C[a1] for a1 in undirected_edges).value)


# should be 1



{0, 1, 2, 3, 4, 5}
{(1, 3), (4, 0), (0, 4), (3, 1), (5, 4), (2, 0), (1, 4), (0, 2), (4, 5), (5, 3), (2, 5), (4, 1), (3, 5), (5, 2)}
minimal objective: 0.9999999999478556


In [28]:
import cvxpy as cp
import numpy as np
import networkx as nx




inputnodes = [0,4]
inputedges = [(0,2),(2,3),(3,4),(4,0),(2,1),(1,4)]

non_member = []
source = [0,1,2,3]
recievers = [[1],[3],[4],[0]]

sessions = [0,1,2,3]

C = {
    (0,2):1,
    (2,3):1,
    (3,4):1,
    (4,0):1,
    (2,1):1,
    (1,4):1,
}


preG = nx.Graph()
preG.add_nodes_from(inputnodes)
preG.add_edges_from(inputedges)
directE = []
for n, nbrs in preG.adj.items():
    for nbr, eattr in nbrs.items():
        directE.append((n,nbr))


G = nx.DiGraph()
G.add_nodes_from(inputnodes)
G.add_edges_from(directE)

undirected_edges = set(inputedges)

vertices = set(np.arange(inputnodes[1]+1))
print(vertices)

edges = set(directE)
print(edges)


# ----- building LP ---------------

fstar_constraints = []
flow_constraints = []
capacity_constraints = []
variable_constraints = []


phi = {(i, j): cp.Variable() for i in sessions for j in recievers[i]}   # f* == incoming reciever flow

fstar_constraints = [(-1 + sum(phi[i,j] for i in sessions for j in recievers[i])) == 0]

mu = {(i, j, a): cp.Variable() for i in sessions for j in recievers[i] for a in edges}   # fija >= 0
psi = {(i, j, a): cp.Variable() for i in sessions for j in recievers[i] for a in edges}   # fija <= fia
xi = {(a): cp.Variable() for a in edges}   # sum(fia) <= c(a)

for i in sessions: 
    for a in edges: 
        variable_constraints += [xi[a] >= 0]      # it loops more times than it needs, change?
        for j in recievers[i]:
            variable_constraints += [psi[i,j,a] >= 0]
            variable_constraints += [mu[i,j,a] >= 0]

rho = {(i, j, v): cp.Variable() for i in sessions for j in recievers[i] for v in vertices - {i} - {j}}   # outfrom non-member == into non-memebr
sigma = {(i, j): cp.Variable() for i in sessions for j in recievers[i]}   # into source == 0
tau = {(i, j): cp.Variable() for i in sessions for j in recievers[i]}   # outfrom reciever == 0

for i in sessions:
    for j in recievers[i]:
        non_member = vertices - {i} - {j}
        for a in edges:
            equation = - mu[i,j,a] + psi[i,j,a]
            if a[1] in non_member:
                equation += rho[i,j,a[1]]
            if a[0] in non_member:
                equation -= rho[i,j,a[0]]
            if a[1] == int(source[i]):
                equation += sigma[i,j]
            if a[0] == int(j):
                equation += tau[i,j]
            if a[1] == int(j):
                equation -= phi[i,j]
                
            flow_constraints += [equation == 0]
        

for a in edges:
    psi_sum = sum(psi[i,j,a] for i in sessions for j in recievers[i])
    flow_constraints += [- psi_sum + xi[a] == 0]


lambd = {a: cp.Variable() for a in edges}   
nu = {a1: cp.Variable() for a1 in undirected_edges}   # fia <= c(a)

for a1 in undirected_edges:
    a2 = (a1[1], a1[0]) 
    equation1 = nu[a1]
    equation2 = nu[a1]
    

    equation1 -= (xi[a1] + lambd[a1])
    equation2 -= (xi[a2] + lambd[a2])

    variable_constraints += [lambd[a1] >= 0]
    variable_constraints += [lambd[a2] >= 0]

    capacity_constraints += [equation1 == 0]
    capacity_constraints += [equation2 == 0]
    

objective = cp.Minimize(sum(nu[a1]*C[a1] for a1 in undirected_edges))

constraints = fstar_constraints + flow_constraints + capacity_constraints + variable_constraints

problem = cp.Problem(objective, constraints)
problem.solve()

print("minimal objective:", sum(nu[a1]*C[a1] for a1 in undirected_edges).value)



# should be 0.75



{0, 1, 2, 3, 4}
{(4, 0), (0, 4), (2, 1), (3, 4), (4, 3), (1, 2), (2, 0), (1, 4), (2, 3), (0, 2), (3, 2), (4, 1)}
minimal objective: 0.9999999986341751
